This is the third attempt at Time Series Forecasting, this time with the goal to set up a live feed

I'll start by moving over my old code (and maybe making it prettier? I think we can skip over the plotting for now)

This notebook was written based on this tutorial
https://www.kaggle.com/code/someadityamandal/bitcoin-time-series-forecasting/notebook

Additional info to the last iteration:
https://www.youtube.com/watch?v=P_SIZDsI3Ro
https://www.youtube.com/watch?v=TF2Nx_ifmrU

We'll do plotting and whatnot towards the bottom. Those'll be their own sets of imports

In [2]:
!pip install chart_studio

import numpy as np
import pandas as pd
import keras.layers as layers
import keras.models as models

import datetime, pytz
from sklearn.preprocessing import MinMaxScaler

import os
print(os.listdir('./data'))

2024-11-13 17:39:53.864110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 17:39:53.934343: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 17:39:54.331290: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 17:39:54.601833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 17:39:54.802233: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

['coinbaseUSD.csv', 'bitstampUSD.csv']


The question is now, how do we take old data and new data into account?
I /think/ that the vibe is that we train a model on the historic data, then we feed the new data into it.
So we'll make the model, then feed new data into it, then plot.
That's the goal for tonight.

In [3]:
def dateparse (time_in_secs):
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))
# That seriously feels like one heck of a line.

data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)

data.info()
print(data.head())


/tmp/ipykernel_829/4154642391.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2099760 entries, 0 to 2099759
Data columns (total 8 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Timestamp          datetime64[ns, UTC]
 1   Open               float64            
 2   High               float64            
 3   Low                float64            
 4   Close              float64            
 5   Volume_(BTC)       float64            
 6   Volume_(Currency)  float64            
 7   Weighted_Price     float64            
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 128.2 MB
                  Timestamp   Open   High    Low  Close  Volume_(BTC)  \
0 2014-11-30 23:33:00+00:00  300.0  300.0  300.0  300.0          0.01   
1 2014-11-30 23:34:00+00:00    NaN    NaN    NaN    NaN           NaN   
2 2014-11-30 23:35:00+00:00    NaN    NaN    NaN    NaN           NaN   
3 2014-11-30 23:36:00+00:00    NaN    NaN    NaN    NaN           NaN   
4 2014-11-30 23:3

In [4]:
# Now for some preprocessing
data['Volume_(BTC)'].fillna(value=0, inplace=True)
data['Volume_(Currency)'].fillna(value=0, inplace=True)
data['Weighted_Price'].fillna(value=0, inplace=True)

data['Open'].fillna(method='ffill', inplace=True)
data['High'].fillna(method='ffill', inplace=True)
data['Low'].fillna(method='ffill', inplace=True)
data['Close'].fillna(method='ffill', inplace=True)

print(data.head())

                  Timestamp   Open   High    Low  Close  Volume_(BTC)  \
0 2014-11-30 23:33:00+00:00  300.0  300.0  300.0  300.0          0.01   
1 2014-11-30 23:34:00+00:00  300.0  300.0  300.0  300.0          0.00   
2 2014-11-30 23:35:00+00:00  300.0  300.0  300.0  300.0          0.00   
3 2014-11-30 23:36:00+00:00  300.0  300.0  300.0  300.0          0.00   
4 2014-11-30 23:37:00+00:00  300.0  300.0  300.0  300.0          0.00   

   Volume_(Currency)  Weighted_Price  
0                3.0           300.0  
1                0.0             0.0  
2                0.0             0.0  
3                0.0             0.0  
4                0.0             0.0  


/tmp/ipykernel_829/3773503921.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume_(BTC)'].fillna(value=0, inplace=True)
/tmp/ipykernel_829/3773503921.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

Wow those sure are a lot of warnings
Especially for Future Ace to figure out

In [5]:
start = datetime.datetime(2015, 1, 1, 0, 0, 0, 0, pytz.UTC)
end = datetime.datetime(2018, 11, 11, 0, 0, 0, 0, pytz.UTC)

weekly_rows = data[(data['Timestamp'] >= start) & (
    data['Timestamp'] <= end)].groupby(
        [pd.Grouper(key='Timestamp', freq='W-MON')]
    ).first().reset_index()

print(weekly_rows.head())
# Idk if that made things look any cleaner.
# I think I did well
# But it also needs to run.

                  Timestamp    Open    High     Low   Close  Volume_(BTC)  \
0 2015-01-12 00:00:00+00:00  360.00  360.00  360.00  360.00         0.010   
1 2015-01-19 00:00:00+00:00  260.00  260.00  260.00  260.00         0.000   
2 2015-01-26 00:00:00+00:00  225.51  225.51  225.51  225.51         0.000   
3 2015-02-02 00:00:00+00:00  259.99  259.99  259.99  259.99         0.300   
4 2015-02-09 00:00:00+00:00  239.28  239.50  239.28  239.50        20.675   

   Volume_(Currency)  Weighted_Price  
0           3.600000      360.000000  
1           0.000000        0.000000  
2           0.000000        0.000000  
3          77.997000      259.990000  
4        4951.178693      239.476599  


Ok nice! Let's consider our data created.
Looking down the pipe I'm genuinely kinda worried about it getting live data

In [6]:
data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)

data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='ffill', inplace=True)

/tmp/ipykernel_829/3787833246.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)
/tmp/ipykernel_829/3787833246.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
/tmp/ipykernel_829/3787833246.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = d

In [7]:
# Split that data
split_date = '25-Jun-2018'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [9]:
# More Preprocessing
training_set = data_train.values
training_set = np.reshape(training_set, (len(training_set), 1))

sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

In [10]:
# Here's the fun part! We're building the model!
# I don't think I'll change anything other than add a layer or two
# Maybe it'll help with the overfitting
# Probably not but worth a try
# We're also swapping sigmoid for relu

model = models.Sequential()

model.add(layers.LSTM(128, activation='relu', input_shape=(1, 1)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
625/625 - 6s - 10ms/step - loss: 0.0117
Epoch 2/100
625/625 - 5s - 7ms/step - loss: 0.0076
Epoch 3/100
625/625 - 2s - 4ms/step - loss: 0.0079
Epoch 4/100
625/625 - 2s - 4ms/step - loss: 0.0075
Epoch 5/100
625/625 - 2s - 4ms/step - loss: 0.0075
Epoch 6/100
625/625 - 2s - 4ms/step - loss: 0.0073
Epoch 7/100
625/625 - 3s - 4ms/step - loss: 0.0078
Epoch 8/100
625/625 - 3s - 4ms/step - loss: 0.0073
Epoch 9/100
625/625 - 2s - 4ms/step - loss: 0.0081
Epoch 10/100
625/625 - 2s - 4ms/step - loss: 0.0075
Epoch 11/100
625/625 - 3s - 4ms/step - loss: 0.0074
Epoch 12/100
625/625 - 3s - 4ms/step - loss: 0.0073
Epoch 13/100
625/625 - 3s - 5ms/step - loss: 0.0076
Epoch 14/100
625/625 - 3s - 4ms/step - loss: 0.0073
Epoch 15/100
625/625 - 2s - 4ms/step - loss: 0.0078
Epoch 16/100
625/625 - 2s - 4ms/step - loss: 0.0075
Epoch 17/100
625/625 - 2s - 4ms/step - loss: 0.0074
Epoch 18/100
625/625 - 3s - 4ms/step - loss: 0.0077
Epoch 19/100
625/625 - 3s - 4ms/step - loss: 0.0075
Epoch 20/100
625/625

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,075 (781.55 KB)

 Trainable params: 66,691 (260.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,384 (521.04 KB)

    Ok so next we would grab some inputs and then run predictions. Here's where we step in to start figuring out how to do a live data feed

    For your memory, this is the video we're following right now:
    https://www.youtube.com/watch?v=P_SIZDsI3Ro

None of that worked out well because the tutorial from 1 year ago is outdated I guess. Time to figure something else out.

Code Graveyard lies below this markdown
Careful ye who may approach

In [22]:
# Imports
import json
import websocket

In [23]:
asset = 'btcusdt@kline_lm'

In [24]:
def on_message(ws, message):
    # message = json.loads(message)
    print(message)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print('closed')

def on_open(ws):
    print('opened')

In [25]:
socket = "wss://stream.binance.com:9443/stream?streams=" + asset

In [27]:
ws = websocket.WebSocketApp(socket,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.on_open = on_open
ws.run_forever()

AttributeError: module 'websocket' has no attribute 'WebSocketApp'